# Generating Implementation Results

Here we will programmatically generate implementation results for different instances of our filter designs with Vivado.

We'll do a parameter sweep over:
  * parallelisms
  * number of taps
  * type of coefficient set (i.e. minimum/non-linear phase, types I/II/III/IV, or half-band)
  * and filter structure (our FFA + MCM, our Polyphase + MMC, and LogiCORE FIR Compiler)

Representations of the different coefficient sets are defined in `coeffs.py` and representations and implementation functions for each filter structure are defined in `filters.py`.

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from math import *

from coeffs import ResponseType, CoeffsMinPhase, CoeffsType1Pad, CoeffsType2, CoeffsHalfBand
from filters import SsrFilter, FfaMcmFilter, PolyMcmFilter

Let's define a few lists of constructors for each coefficient style and each filter structure. After that, all we need is a few nested loops around our `fir.impl()` call. This will take a _while_... somewhere in the order of a whole weekend!

In [ ]:
coeff_constrs = [
    lambda taps : CoeffsMinPhase(ResponseType.LP, [0,0.3,0.4,1.0], taps, 16),
    lambda taps : CoeffsType1Pad(ResponseType.LP, [0,0.3,0.4,1.0], taps, 16),
    lambda taps :    CoeffsType2(ResponseType.LP, [0,0.3,0.4,1.0], taps, 16),
    lambda taps : CoeffsHalfBand(taps, 16)
]

fir_constrs = [
    lambda par, taps, ws, out_dir :     SsrFilter(par, 16, ws, 775.0, out_dir),
    lambda par, taps, ws, out_dir : PolyMcmFilter(par, 16, ws, 775.0, out_dir),
    lambda par, taps, ws, out_dir :  FfaMcmFilter(par, 16, ws, 775.0, out_dir),
]

results = []

for par in [2,4,8,16]:
    for taps in [2,4,8,16,32,64,128]:
        if par > taps:
            continue
        for f_coef in coeff_constrs:
            for f_fir in fir_constrs:
                ws = f_coef(taps)
                fir = f_fir(par, taps, ws, './outputs')
                res = fir.impl()
                print(res)
                results.append(res)

So we've generated everything we need! All the Vivado runs will be in the `outputs/` folder. While we have the parsed results in memory though, let's save them as a CSV for posterity.

In [ ]:
df = pd.DataFrame(results)

df.to_csv('outputs/full_impl.csv')

And of course, if you want to load the results back in again, you can run `df = pd.read_csv('outputs/full_impl.csv')`.

Below we will plot our results for CLB usage, DSP usage, and estimated achieved frequency.

In [ ]:
display(px.line(df, x='taps', facet_col='parallelism', facet_row='coeff_class',y='clbs', color='structure', height=1000))

In [ ]:
display(px.line(df, x='taps', facet_col='parallelism', facet_row='coeff_class',y='dsps', color='structure', height=1000))

In [ ]:
display(px.line(df, x='taps', facet_col='parallelism', facet_row='coeff_class',y='fclk_actual', color='structure', height=1000))

Let's export a flattened CSV file for each of these metrics. This is a little bit easier to work with in LaTeX/pgfplots than the earlier CSV.

In [ ]:
for (field, scale) in [('dsps', 100/4272), ('clbs', 100/53160), ('fclk_actual', 1)]:
    df_export = df.copy()
    df_export[field] = df_export[field]*scale
    df_export = df_export.pivot(index=['taps'], columns=['structure','parallelism', 'coeff_class'], values=field)
    df_export.columns = ['_'.join([str(c) for c in col]) for col in df_export.columns.values]

    df_export.to_csv(f'outputs/summary_{field}.csv', index = True, header=True)